In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
from lime import lime_tabular
import string
import nltk
from nltk.corpus import stopwords
import random
from sklearn.naive_bayes import MultinomialNB


In [36]:
# Data Preprocessing

def preprocessing(fake=pd.read_csv("True.csv"),true=pd.read_csv("True.csv")):
    fake['target'] = 'fake'
    true['target'] = 'true'
    data = pd.concat([fake, true]).reset_index(drop = True)
    data.drop(["date"],axis=1,inplace=True)
    data.drop(["title"],axis=1,inplace=True)
    data['text'] = data['text'].apply(lambda x: x.lower())

    def punctuation_removal(text):
        all_list = [char for char in text if char not in string.punctuation]
        clean_str = ''.join(all_list)
        return clean_str

    data['text'] = data['text'].apply(punctuation_removal)
    nltk.download('stopwords')
    stop = stopwords.words('english')
    data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    #print('data lenght = ',len(data))
    #print('lenfake = ',lenfake)
    #print('lentrue = ',lentrue)
   
    return(data)

d=preprocessing()
print(d)

[nltk_data] Downloading package stopwords to /Users/evan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                                    text       subject target
0      washington reuters head conservative republica...  politicsNews   fake
1      washington reuters transgender people allowed ...  politicsNews   fake
2      washington reuters special counsel investigati...  politicsNews   fake
3      washington reuters trump campaign adviser geor...  politicsNews   fake
4      seattlewashington reuters president donald tru...  politicsNews   fake
...                                                  ...           ...    ...
42829  brussels reuters nato allies tuesday welcomed ...     worldnews   true
42830  london reuters lexisnexis provider legal regul...     worldnews   true
42831  minsk reuters shadow disused sovietera factori...     worldnews   true
42832  moscow reuters vatican secretary state cardina...     worldnews   true
42833  jakarta reuters indonesia buy 11 sukhoi fighte...     worldnews   true

[42834 rows x 3 columns]


In [14]:
def dataproc(data,nbfake,nbtrue,lenfake,lentrue):
    #print(data)
    if nbfake>=(lenfake-1):
        print('Not enough fake news in dataset')
    if nbtrue>=(lentrue-1):
        print('Not enough true news in dataset')
    print(len(data))
    a=random.sample(range(0, lenfake), nbfake)
    b=random.sample(range(lenfake+1, len(data)-1), nbtrue)
    #print(b)
    print(' b = ',b)
    d=data.iloc[a][:]
    print(" a = ", a)

    print("data = ",data)
    e=data.iloc[b][:]
    
    data2=pd.concat([d,e], axis=0)

    X_train,X_test,y_train,y_test = train_test_split(data2['text'],data2.target,test_size=0.2,random_state=42)

    return(X_train,X_test,y_train,y_test)

#data2=dataproc(preprocessing(),nbfake,nbtrue)
#print(data2[0])


In [15]:
'''def subjects(data):
    print(data.groupby(['subject'])['text'].count())
    data.groupby(['subject'])['text'].count().plot(kind="bar")
    plt.show()

subjects(dataproc(data2,13,14))'''

'def subjects(data):\n    print(data.groupby([\'subject\'])[\'text\'].count())\n    data.groupby([\'subject\'])[\'text\'].count().plot(kind="bar")\n    plt.show()\n\nsubjects(dataproc(data2,13,14))'

In [16]:
def vecto(X_train,X_test):
    
    X_t=pd.concat([X_train,X_test])
    #print('X_t = ',X_t.shape)
    ltotal=len(X_t)
    #print(X_t)
    vectorizer = TfidfVectorizer(lowercase=False)
    v = vectorizer.fit_transform(X_t)

    features=vectorizer.get_feature_names_out()
    ltrain=len(X_train)
    vtrain=v[0:ltrain][:]
    vtest=v[ltrain:][:]
    vtrain=scipy.sparse.csr_matrix.toarray(vtrain)
    vtest=scipy.sparse.csr_matrix.toarray(vtest)

    return(vtrain,vtest,vectorizer,features)
    

#print(vecto(data2[0],data2[1]))

In [17]:

def classifLogReg(vtrain,vtest,y_train,y_test):
    mo=LogisticRegression()
    mo.fit(vtrain,y_train)
    m=mo.predict(vtest)
    m=np.array(m)
    y_test=np.array(y_test)
    #print(y_test)
    #print(m)

    #print("accuracy: {}%".format(round(accuracy_score(y_test, m)*100,2)))
    return(round(accuracy_score(y_test, m)*100,2))
print(classifLogReg)

<function classifLogReg at 0x7fe5497a0700>


In [18]:

def classifNaiveBayes(vtrain,y_train,vectorizer,X_test,features):
    mo=MultinomialNB()
    mo.fit(vtrain,y_train)

    i=X_test.index
    j=i[1]

    o=pd.Series(X_test[j])
    o=vectorizer.transform(o)
    o=scipy.sparse.csr_matrix.toarray(o)
    o=o[0]

    explainer = lime_tabular.LimeTabularExplainer(vtrain,mode="classification",categorical_features=features,feature_names=features)
    exp=explainer.explain_instance(data_row=o,predict_fn=mo.predict_proba, num_features=20) #labels=features
    exp.as_pyplot_figure()
    from matplotlib import pyplot as plt
    plt.tight_layout()
    exp.show_in_notebook()
    return(j,exp)

In [19]:
'''print(features)
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(8,6))
    plt.barh(range(len(mo.coef_[0])), mo.coef_[0], color=["red" if coef<0 else "green" for coef in mo.coef_[0]])
    plt.yticks(range(len(mo.coef_[0])), features);
    plt.title("Weights")
    '''

'print(features)\nwith plt.style.context("ggplot"):\n    fig = plt.figure(figsize=(8,6))\n    plt.barh(range(len(mo.coef_[0])), mo.coef_[0], color=["red" if coef<0 else "green" for coef in mo.coef_[0]])\n    plt.yticks(range(len(mo.coef_[0])), features);\n    plt.title("Weights")\n    '

In [20]:
'''def limenaivebayes(nbfake,nbtrue):
    data,lenfake,lentrue=preprocessing()
    
    x1=dataproc(data,nbfake,nbtrue,lenfake,lentrue)
    X_train=x1[0]
    X_test=x1[1]
    y_train=x1[2]
    y_test=x1[3]
    vtrain,vtest,vectorizer,features=vecto(X_train,X_test)
    #classifNaiveBayes(vtrain,y_train,vectorizer,X_test,features)
    j,exp=classifNaiveBayes(vtrain,y_train,vectorizer,X_test,features)
    #print('Texte classé = ',X_test[j])

    outputs = open("outputs.txt", "w")
    outputs.write("Texte classé = ")
    outputs.write(X_test[j])
    #outputs.write(exp)
    outputs.write("/n")
    outputs.close()
    
limenaivebayes(5,5)

'''


'def limenaivebayes(nbfake,nbtrue):\n    data,lenfake,lentrue=preprocessing()\n    \n    x1=dataproc(data,nbfake,nbtrue,lenfake,lentrue)\n    X_train=x1[0]\n    X_test=x1[1]\n    y_train=x1[2]\n    y_test=x1[3]\n    vtrain,vtest,vectorizer,features=vecto(X_train,X_test)\n    #classifNaiveBayes(vtrain,y_train,vectorizer,X_test,features)\n    j,exp=classifNaiveBayes(vtrain,y_train,vectorizer,X_test,features)\n    #print(\'Texte classé = \',X_test[j])\n\n    outputs = open("outputs.txt", "w")\n    outputs.write("Texte classé = ")\n    outputs.write(X_test[j])\n    #outputs.write(exp)\n    outputs.write("/n")\n    outputs.close()\n    \nlimenaivebayes(5,5)\n\n'

In [53]:
#data=preprocessing()
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer=SentimentIntensityAnalyzer()
print("news = ",data['text'])
sentiment=[]
for news in data['text'][0:5]:
    x=analyzer.polarity_scores(news)['neg']
    sentiment=sentiment+[x]
print(sentiment[0])

'''
positive sentiment: compound score >= 0.05
neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
negative sentiment: compound score <= -0.05
'''



news =  0        washington reuters head conservative republica...
1        washington reuters transgender people allowed ...
2        washington reuters special counsel investigati...
3        washington reuters trump campaign adviser geor...
4        seattlewashington reuters president donald tru...
                               ...                        
42829    brussels reuters nato allies tuesday welcomed ...
42830    london reuters lexisnexis provider legal regul...
42831    minsk reuters shadow disused sovietera factori...
42832    moscow reuters vatican secretary state cardina...
42833    jakarta reuters indonesia buy 11 sukhoi fighte...
Name: text, Length: 42834, dtype: object
0.064


'\npositive sentiment: compound score >= 0.05\nneutral sentiment: (compound score > -0.05) and (compound score < 0.05)\nnegative sentiment: compound score <= -0.05\n'